In [1]:
import pandas as pd
import utils
import numpy as np
from tqdm import tqdm
import collections
import time
import copy
import os
import datetime
from datetime import date
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import calendar
from pathlib import Path
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from email.mime.text import MIMEText
import smtplib
home = str(Path.home())
t1 = time.time()
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
# import seaborn as sns

In [2]:
# Snowflake connector dbt_jferragut
engine = utils.create_snowflake_engine(TAG = 'DIRECT PROFITABILITY')
connection = engine.connect()

# PyCaret

In [4]:
from pycaret.classification import *

In [8]:
encoded.columns

Index(['bookingreference', 'booking_timestamp', 'lead_time_days',
       'length_of_stay', 'top_fraud_agency_flag', 'total_ko_web',
       'total_ko_api', 'top_fraud_destination_flag', 'adr', 'hotel_cost_max',
       'hotel_cost_max_year', 'cluster_cost_max', 'dicrepancy_cm',
       'dicrepancy_cy', 'dicrepancy_destcat', 'is_fraud',
       'PAYMENTTYPE_BookNowPayLater', 'PAYMENTTYPE_Credit',
       'PAYMENTTYPE_PayWithCreditCard', 'CHANNEL_evolution',
       'CHANNEL_evolution_hotelapi', 'CHANNEL_travelstack'],
      dtype='object')

In [9]:
encoded[encoded['is_fraud']==False]

,bookingreference,booking_timestamp,lead_time_days,length_of_stay,top_fraud_agency_flag,total_ko_web,total_ko_api,top_fraud_destination_flag,adr,hotel_cost_max,...,dicrepancy_cm,dicrepancy_cy,dicrepancy_destcat,is_fraud,PAYMENTTYPE_BookNowPayLater,PAYMENTTYPE_Credit,PAYMENTTYPE_PayWithCreditCard,CHANNEL_evolution,CHANNEL_evolution_hotelapi,CHANNEL_travelstack
0,256-13030221,2024-07-31 17:28:07.161,55,1,False,0,0,False,198.212040,789.97,...,-0.084344,0.133547,0.129156,False,True,False,False,False,True,False
1,256-9830863,2023-11-20 03:11:38.026,306,4,False,0,0,False,245.197027,789.97,...,0.132707,0.402248,0.396816,False,True,False,False,False,True,False
2,256-13030220,2024-07-31 17:28:03.716,42,2,False,0,0,False,193.435917,789.97,...,-0.106408,0.106233,0.101948,False,False,False,False,False,True,False
3,256-13183437,2024-09-03 19:35:19.494,21,1,False,0,0,False,212.329328,789.97,...,-0.019128,0.214282,0.209578,False,False,False,True,True,False,False
4,256-13405142,2024-10-22 16:43:54.578,318,3,False,0,0,False,209.020000,789.97,...,-0.034416,0.195356,0.190726,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2897691,102-17404908,2024-06-06 23:09:20.816,84,3,False,0,0,False,434.240005,1715.75,...,0.264163,0.136754,0.702902,False,False,True,False,False,True,False
2897692,102-17552692,2024-07-08 17:59:56.124,31,4,False,0,0,False,2225.713268,1715.75,...,5.479515,4.826475,7.728287,False,False,False,False,False,True,False
2897693,102-16872457,2024-02-19 09:33:57.989,181,3,False,0,0,False,524.695214,1715.75,...,0.527497,0.373548,1.057628,False,False,True,False,False,True,False
2897694,102-17511521,2024-06-29 23:26:19.531,55,2,False,0,0,False,323.952039,1715.75,...,-0.056908,-0.151958,0.270400,False,False,False,True,False,True,False


In [12]:
encoded_sample, _ =  train_test_split(encoded, test_size=0.99655, random_state=123, stratify=encoded['is_fraud'])

In [18]:
from imblearn.over_sampling import SMOTE

In [19]:
# Initial configuration
clf = setup(
    data=encoded_sample, 
    target='is_fraud',  # Variable objetivo que indica fraude (0 o 1)
    numeric_features=['lead_time_days',
       'length_of_stay', 'top_fraud_agency_flag', 'total_ko_web',
       'total_ko_api', 'top_fraud_destination_flag', 'adr', 'hotel_cost_max',
       'hotel_cost_max_year', 'cluster_cost_max', 'dicrepancy_cm',
       'dicrepancy_cy', 'dicrepancy_destcat', 
       'PAYMENTTYPE_BookNowPayLater', 'PAYMENTTYPE_Credit',
       'PAYMENTTYPE_PayWithCreditCard', 'CHANNEL_evolution',
       'CHANNEL_evolution_hotelapi', 'CHANNEL_travelstack'],  # Variables numéricas
    session_id=42,  # Fijar semilla para reproducibilidad
    fix_imbalance=True,  # Manejo automático del desbalanceo
    fix_imbalance_method= SMOTE(k_neighbors=1)
    # silent=True  # No pedir confirmación al ejecutar
)

# =============================
# Comparar múltiples modelos
# =============================
# Esto entrena y compara múltiples modelos (como LightGBM, Random Forest, etc.)
best_model = compare_models(sort='F1')  # Ordenar por F1-score (clave en problemas desbalanceados)

# =============================
# Entrenar el mejor modelo
# =============================
# Una vez seleccionado el mejor modelo, lo entrenamos
final_model = finalize_model(best_model)

# =============================
# Predicciones en el conjunto de prueba
# =============================
# PyCaret automáticamente divide los datos en entrenamiento y prueba
predictions = predict_model(final_model)

# Mostrar un resumen de las métricas
print(predictions.head())

,Description,Value
0,Session id,42
1,Target,is_fraud
2,Target type,Binary
3,Original data shape,"(9997, 22)"
4,Transformed data shape,"(16988, 24)"
5,Transformed train set shape,"(13988, 24)"
6,Transformed test set shape,"(3000, 24)"
7,Numeric features,19
8,Date features,1
9,Categorical features,1


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
svm,SVM - Linear Kernel,0.9757,0.2082,0.2000,0.0271,0.0467,0.0463,0.0711,0.0640
lr,Logistic Regression,0.9974,0.2016,0.1000,0.0250,0.0400,nan,0.0499,1.2170
knn,K Neighbors Classifier,0.9973,0.1999,0.1000,0.0200,0.0333,0.0332,0.0446,0.0950
nb,Naive Bayes,0.8601,0.1667,0.0000,0.0000,0.0000,-0.0005,-0.0010,0.0550
dt,Decision Tree Classifier,0.9996,0.1500,0.0000,0.0000,0.0000,nan,0.0000,0.0670
ridge,Ridge Classifier,0.9994,0.1913,0.0000,0.0000,0.0000,nan,-0.0001,0.0560
rf,Random Forest Classifier,0.9996,0.2483,0.0000,0.0000,0.0000,nan,0.0000,0.2400
ada,Ada Boost Classifier,0.9996,0.1500,0.0000,0.0000,0.0000,nan,0.0000,0.0740
gbc,Gradient Boosting Classifier,0.9996,0.0705,0.0000,0.0000,0.0000,nan,0.0000,1.5890
lda,Linear Discriminant Analysis,0.9996,0.1500,0.0000,0.0000,0.0000,nan,0.0000,0.1000


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,SVM - Linear Kernel,0.8880,0.6941,0.5000,0.0030,0.0059,0.0046,0.0318


        bookingreference       booking_timestamp  lead_time_days  \
582873        71-5773760 2024-06-26 21:08:45.960              39   
113486       436-5988171 2024-02-16 00:08:28.223              93   
277641       321-6726583 2023-12-18 03:22:57.219               0   
1062316       71-5887199 2024-09-13 22:56:39.158              59   
2225362     102-17818303 2024-09-10 12:42:06.671              28   

         length_of_stay  top_fraud_agency_flag  total_ko_web  total_ko_api  \
582873                2                  False             0             0   
113486                3                  False             0             0   
277641                1                  False             0             0   
1062316               3                  False             0             0   
2225362               1                  False             0             0   

         top_fraud_destination_flag         adr  hotel_cost_max  ...  \
582873                        False  222.339600   